<a href="https://colab.research.google.com/github/vvmnnnkv/syft-js-worker/blob/master/Syft%20Web%20Client%20Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Syft Web Worker 🕸

This notebook demonstrates the [project](https://github.com/vvmnnnkv/syft-js-worker) created for Udacity's [Secure and Private AI](https://www.udacity.com/course/secure-and-private-ai--ud185) challenge course.

[PySyft](https://www.openmined.org/) is the library that (among other things) provides PyTorch wrapper for distributed operations and enables privacy and security applications such as federated learning, multi-party computation, etc.

This project explores possibility to have PySyft worker running in the web browser, which potentially expands PySyft  functionality to a very large user-base. The back-end for tensor operations in browser is [tf.js](https://www.tensorflow.org/js).

The current project's state is 'proof of concept' where tensors can be moved to/from browser and a limited set of tensor operations is possible. Further direction is to try to use web client for federated learning.

In [0]:
# Install PySyft & Grid is not already installed
import importlib, sys
if importlib.util.find_spec("syft") is None:
  !pip install syft

if importlib.util.find_spec("grid") is None:
  !git clone https://github.com/OpenMined/Grid; cd Grid; pip install -Ur requirements.txt; python setup.py install
  # make it available for import w/o colab runtime restart 🙌
  sys.path.insert(0, "Grid/build/lib")


In [2]:
# Imports
import syft as sy
from syft.frameworks.torch.tensors.decorators import LoggingTensor
import sys
import torch
from grid.workers.socketio_client import WebsocketIOClientWorker

# Updated numpy serializer that returns array of bytes instead of string
# this allows javascript msgpack to correctly unpack it to array of bytes instead of utf16 string
def numpy_tensor_serializer(tensor: torch.Tensor) -> bin:
    return list(sy.torch_serde.numpy_tensor_serializer(tensor))

# Set numpy serialiser and disable compression
sy.torch_serde._serialize_tensor = numpy_tensor_serializer
sy.torch_serde._deserialize_tensor = sy.torch_serde.numpy_tensor_deserializer
sy.serde.serde._apply_compress_scheme = sy.serde.apply_no_compression

hook = sy.TorchHook(torch)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

## Connect PySyft Worker
SocketIO [server](https://github.com/vvmnnnkv/syft-js-worker/blob/master/run_socketio_server.py) running at **syft-socketio-server.herokuapp.com** is a proxy between WebsocketIOClientWorker in this notebook and javascript worker in the browser.

First, let's create and connect PySyft worker client in the notebook.

In [0]:
# web worker proxy
host = 'syft-socketio-server.herokuapp.com'
port = 443


bob = WebsocketIOClientWorker(hook, host=host, port=port, id="bob", verbose=True)
# workaround to use https
bob.uri = bob.uri.replace('http:', 'https:')
bob.connect()

## Connect Javacript Worker

Open https://vvmnnnkv.github.io/syft-js-worker and make sure it connects.

Now, we can send tensors and commands to worker and get results back. You should see activity in the browser as it happens.

In [4]:
x = torch.tensor([1., 2, 3, 4, 5])
x_ptr = x.send(bob)
x_ptr

(Wrapper)>[PointerTensor | me:59220251427 -> bob:49232486277]

In [5]:
y = torch.tensor([10., 20, 30, 40, 50])
y_ptr = y.send(bob)
y_ptr

(Wrapper)>[PointerTensor | me:88328616582 -> bob:34391114985]

In [6]:
z_ptr = x_ptr + y_ptr
z = z_ptr.get()
z

tensor([11., 22., 33., 44., 55.])

In [7]:
z_ptr = x_ptr * y_ptr
z = z_ptr.get()
z

tensor([ 10.,  40.,  90., 160., 250.])

In [8]:
# Simplest linear layer
X_ptr = torch.randn(10, 3).send(bob)
w_ptr = torch.randn(3, 1).send(bob)
b_ptr = torch.randn(1).send(bob)

z_ptr = X_ptr @ w_ptr + b_ptr
z = z_ptr.get()
z

tensor([[-1.6811],
        [ 3.7574],
        [ 2.5846],
        [ 3.2510],
        [ 0.5770],
        [ 1.3945],
        [-0.8392],
        [ 0.8744],
        [ 0.7503],
        [-2.3768]])